<a href="https://colab.research.google.com/github/Ismail-therap/NLP-Project-Predicting-review-in-1-to-5-star-/blob/main/CNN_5%2C_3_and_2_category_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Import libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

In [68]:
import pandas as pd

url='https://drive.google.com/file/d/1-vBpWV0H0cxX5jSeHlQIKXIiosMKwq2t/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
data = pd.read_csv(dwn_url)
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,0tGuFBrhM03I6TCLxDQhyg,Dancing Goats Coffee Bar,419 W Ponce De Leon Ave,Decatur,GA,30030,33.775805,-84.303269,4.0,358,1,"{'RestaurantsPriceRange2': '1', 'WiFi': ""'free...","Food, Coffee & Tea, Restaurants, Donuts, Cafes","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",IqUD0yoRq66PqnbSvRb5JA,1,OMG!! Hate! This place. Rude to me. On so many...,2018-10-06 14:34:51
1,498F3rnpng6qv53S9diTSw,Boston Market,"4607 Braker Ln, Ste 100",Austin,TX,78759,30.399832,-97.744997,3.0,102,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","American (Traditional), Caterers, Chicken Shop...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",n2eAlOnBo1St1p7MWqBFmg,1,Why bother being open on Thanksgiving if you c...,2019-11-28 20:02:11
2,7qBi3FMT2Oz4Jj1hCWwawA,Bea's Nail Salon,75 Blossom Ct,Boston,MA,02114,42.364891,-71.066818,3.0,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Nail Salons, Beauty & Spas","{'Monday': '9:30-19:0', 'Tuesday': '9:30-19:0'...",pHTrbVN4lzTlzqqnWEJYGw,1,"The fact that on a Saturday afternoon this ""sa...",2009-07-11 19:59:46
3,K4X4nJVp0BcSn9Cvoavk8A,T'Kilas Tacos & More,1248 Clairmont Rd,Decatur,GA,30030,33.789815,-84.307511,3.0,115,0,"{'GoodForKids': 'True', 'BikeParking': 'False'...","Mexican, Restaurants","{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",Jfc8qYR3uEHIpU0nKzpObg,1,"I came to Los Loros in search of the ""strong, ...",2008-09-08 21:34:50
4,D6U1_06gxciRs1NoYjwhKQ,ESPN Club - Temporarily Unavailable,2101 N Epcot Resorts Blvd,Orlando,FL,32836,28.368008,-81.555289,2.5,303,1,"{'GoodForKids': 'True', 'Caters': 'False', 'Re...","Hotels, American (New), Event Planning & Servi...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-23:0', ...",Dw_oka5MVWOsaJCncfIXZQ,1,My fries were stale and my cheesesteak was mos...,2018-01-14 01:41:50


# Five category

In [69]:
from sklearn.model_selection import train_test_split

data['five_star'] = data['review_stars'] - 1
review = data['text'].values
label = data['five_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[ 550  417  242    1 1543    1 1006   24   86 1265 1373    2 3166    1
 1006   24  528   86    1   68  212   13 2041    1 1006   24    1  143
   16 4350 1633 2921 2782  515 4428 4429 1606 3014    1  268   24   88
   21  474    4  256    2   15 1945    8   10    4 1059  905    9   21
 1846   31    7   47    7   12 1441    5   42 3811 1327  356 1595    7
  268  206  156    2 1658    6 1966  218    3  152   69 2254   15   74
 2198  715 1666  843 3445   49  534 3445   75 2921 2782  515 4428 4429
 1606 3014  178   44    3  289   17  102    9    4  230  550 2992   28
   26  550   10  741 2921 2782  515 4428 4429  550    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [70]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 450, 100)          5492700   
                                                                 
 conv1d_22 (Conv1D)          (None, 450, 128)          51328     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 225, 128)         0         
 1D)                                                             
                                                                 
 conv1d_23 (Conv1D)          (None, 225, 64)           32832     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 112, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_24 (Conv1D)          (None, 112, 32)         

In [71]:
model.fit(Xcnn_train, label_train,
                     epochs=5,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/5
586/586 [==============================] - 273s 464ms/step - loss: 1.1550 - accuracy: 0.4701 - val_loss: 0.9457 - val_accuracy: 0.5774
Epoch 2/5
586/586 [==============================] - 275s 469ms/step - loss: 0.8678 - accuracy: 0.6193 - val_loss: 0.9494 - val_accuracy: 0.5890
Epoch 3/5
586/586 [==============================] - 275s 469ms/step - loss: 0.7666 - accuracy: 0.6676 - val_loss: 0.9821 - val_accuracy: 0.5802
Epoch 4/5
586/586 [==============================] - 271s 463ms/step - loss: 0.6688 - accuracy: 0.7126 - val_loss: 1.0255 - val_accuracy: 0.5823
Epoch 5/5
586/586 [==============================] - 282s 481ms/step - loss: 0.5576 - accuracy: 0.7674 - val_loss: 1.1149 - val_accuracy: 0.5814
Training Accuracy: 0.8665
Testing Accuracy:  0.5814


# Three category

In [61]:
import numpy as np

# create a list of our conditions
conditions = [
    (data['review_stars'] <= 2),
    (data['review_stars'] == 3),
    (data['review_stars'] > 3)
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2]

# create a new column and use np.select to assign values to it using our lists as arguments
data['three_star'] = np.select(conditions, values)


# display updated DataFrame
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date,three_star
0,0tGuFBrhM03I6TCLxDQhyg,Dancing Goats Coffee Bar,419 W Ponce De Leon Ave,Decatur,GA,30030,33.775805,-84.303269,4.0,358,1,"{'RestaurantsPriceRange2': '1', 'WiFi': ""'free...","Food, Coffee & Tea, Restaurants, Donuts, Cafes","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",IqUD0yoRq66PqnbSvRb5JA,1,OMG!! Hate! This place. Rude to me. On so many...,2018-10-06 14:34:51,0
1,498F3rnpng6qv53S9diTSw,Boston Market,"4607 Braker Ln, Ste 100",Austin,TX,78759,30.399832,-97.744997,3.0,102,1,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","American (Traditional), Caterers, Chicken Shop...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",n2eAlOnBo1St1p7MWqBFmg,1,Why bother being open on Thanksgiving if you c...,2019-11-28 20:02:11,0
2,7qBi3FMT2Oz4Jj1hCWwawA,Bea's Nail Salon,75 Blossom Ct,Boston,MA,02114,42.364891,-71.066818,3.0,23,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Nail Salons, Beauty & Spas","{'Monday': '9:30-19:0', 'Tuesday': '9:30-19:0'...",pHTrbVN4lzTlzqqnWEJYGw,1,"The fact that on a Saturday afternoon this ""sa...",2009-07-11 19:59:46,0
3,K4X4nJVp0BcSn9Cvoavk8A,T'Kilas Tacos & More,1248 Clairmont Rd,Decatur,GA,30030,33.789815,-84.307511,3.0,115,0,"{'GoodForKids': 'True', 'BikeParking': 'False'...","Mexican, Restaurants","{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",Jfc8qYR3uEHIpU0nKzpObg,1,"I came to Los Loros in search of the ""strong, ...",2008-09-08 21:34:50,0
4,D6U1_06gxciRs1NoYjwhKQ,ESPN Club - Temporarily Unavailable,2101 N Epcot Resorts Blvd,Orlando,FL,32836,28.368008,-81.555289,2.5,303,1,"{'GoodForKids': 'True', 'Caters': 'False', 'Re...","Hotels, American (New), Event Planning & Servi...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-23:0', ...",Dw_oka5MVWOsaJCncfIXZQ,1,My fries were stale and my cheesesteak was mos...,2018-01-14 01:41:50,0


In [62]:
reviews_smallest = data[data["three_star"] == 1]

## sampling negative review:
reviews_neg = data[data["three_star"] == 0]
reviews_neg = reviews_neg.sample(n=reviews_smallest.shape[0])


## sampling positive review:
reviews_pos = data[data["three_star"] == 2]
reviews_pos = reviews_pos.sample(n=reviews_smallest.shape[0])

data = reviews_smallest.append([reviews_neg, reviews_pos])
data.shape

(30000, 19)

In [64]:
from sklearn.model_selection import train_test_split

#data['cnn_star'] = data['three_star'] - 1
review = data['text'].values
label = data['three_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[  19   39   28    1  103   11 1068    2  429   10   13  865  232    3
  161  126 4909   71   20  118  414   93   19  581  160    8   11    4
  839  982  126  513    7 1491 4203   10   33    2    1  109   11  407
   59    5   44    4  221    2  752    1  774   19   25   97    5  181
    1   27    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [65]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


#model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 450, 100)          4315200   
                                                                 
 conv1d_19 (Conv1D)          (None, 450, 128)          51328     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 225, 128)         0         
 1D)                                                             
                                                                 
 conv1d_20 (Conv1D)          (None, 225, 64)           32832     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 112, 64)          0         
 1D)                                                             
                                                                 
 conv1d_21 (Conv1D)          (None, 112, 32)         

In [66]:
model.fit(Xcnn_train, label_train,
                     epochs=5,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/5
352/352 [==============================] - 148s 416ms/step - loss: 0.8342 - accuracy: 0.5841 - val_loss: 0.6711 - val_accuracy: 0.6832
Epoch 2/5
352/352 [==============================] - 146s 416ms/step - loss: 0.6048 - accuracy: 0.7245 - val_loss: 0.6763 - val_accuracy: 0.6965
Epoch 3/5
352/352 [==============================] - 148s 421ms/step - loss: 0.4879 - accuracy: 0.7898 - val_loss: 0.6479 - val_accuracy: 0.7188
Epoch 4/5
352/352 [==============================] - 149s 423ms/step - loss: 0.3684 - accuracy: 0.8492 - val_loss: 0.7197 - val_accuracy: 0.7141
Epoch 5/5
352/352 [==============================] - 147s 418ms/step - loss: 0.2638 - accuracy: 0.8966 - val_loss: 0.8936 - val_accuracy: 0.7033
Training Accuracy: 0.9542
Testing Accuracy:  0.7033


# Two category

In [55]:
import numpy as np

# create a list of our conditions
conditions = [
    (data['review_stars'] <= 2),
    (data['review_stars'] == 3),
    (data['review_stars'] > 3)
    ]

# create a list of the values we want to assign for each condition
values = [0,-1,1]

# create a new column and use np.select to assign values to it using our lists as arguments
data['two_star'] = np.select(conditions, values)


data = data[data.two_star != -1]

# display updated DataFrame
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date,three_star,two_star
14080,MwIazBE9_1yhrLRpyATi5Q,Berkshires at Windsor Gardens by Corcoran Mana...,624 Walpole St,Norwood,MA,02062,42.172698,-71.224265,2.5,38,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Real Estate, Apartments","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",ul8lzfKvMT3TItNwbULaXQ,2,Where to begin...I'm actually embarrassed to b...,2014-03-29 16:08:27,1,0
2851,oyw0rhASxSexoB1xoiFAbg,Bark and Board,2600 Apple Valley Rd NE,Atlanta,GA,30319,33.862477,-84.337267,3.0,41,1,NaN,"Pet Services, Pet Stores, Pet Groomers, Pets, ...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ...",eMa_m3Zb1mys5lQASudxHw,1,"The web cams don't work, so I wasn't able to c...",2016-11-19 13:50:19,1,0
16745,2AORsy2N2kbFX91EXm10Qg,Gran Gusto,90 Sherman St,Cambridge,MA,02140,42.388679,-71.132293,4.0,371,1,"{'OutdoorSeating': 'True', 'RestaurantsTakeOut...","Restaurants, Pizza, Italian","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",09vwFjEOLBhLza6Im_H3pQ,2,"Some good, some bad.\n\nI tried a pizza, the t...",2016-05-11 01:13:22,1,0
5002,P1RqgjZFUy8FaBwET3P6pA,Pollo Tropical,11860 E Colonial Dr,Orlando,FL,32826,28.565949,-81.205686,2.0,75,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Chicken Shop, Latin American, Caribbean, Resta...","{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'...",HaxTzYlNxTinO2VGAxFKtQ,1,"I love Pollo, and get it consistently, but thi...",2020-10-25 00:01:09,1,0
14347,_AXiiZrpuI-YXggCjfhK6Q,Fish Sauce,407 NW 17th Ave,Portland,OR,97209,45.525830,-122.688460,4.0,278,1,"{'Caters': 'True', 'BikeParking': 'True', 'Goo...","Cocktail Bars, Bars, Vietnamese, Restaurants, ...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:0', ...",hYO2BKg8pWIn2FSAhpTBGg,2,First time here and the reason for the low rat...,2014-10-31 02:48:34,1,0


In [57]:
from sklearn.model_selection import train_test_split

#data['cnn_star'] = data['three_star'] - 1
review = data['text'].values
label = data['two_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

#num_classes = 5
#at_label_train = k.utils.to_categorical(label_train, num_classes)
#at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[ 110  395  133 2680   30    1  392  234   11 1347  212  611   30   13
   13   73 1052   52   73   12 2190    9  518 1227  611  112    2 1122
  823   72  868    1 2757  655   44    3 3035  186    1   30   11  313
    2  236  206    2    1  112   11  230    4  115    5  695 1539  623
   61    9   53 2497  112   19    1  268 2630    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [45]:
import tensorflow as tf

embedding_dim = 100
textcnnmodel = Sequential()
textcnnmodel.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(tf.keras.layers.Conv1D(200, 5, activation='relu',padding = 'same'))
#textcnnmodel.add(Dense(100, activation='relu'))
#textcnnmodel.add(tf.keras.layers.GlobalMaxPooling1D())
textcnnmodel.add(tf.keras.layers.Flatten())
textcnnmodel.add(tf.keras.layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
# create the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 450, 100)          3538700   
                                                                 
 conv1d_16 (Conv1D)          (None, 450, 128)          51328     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 225, 128)         0         
 1D)                                                             
                                                                 
 conv1d_17 (Conv1D)          (None, 225, 64)           32832     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 112, 64)          0         
 1D)                                                             
                                                                 
 conv1d_18 (Conv1D)          (None, 112, 32)         

In [59]:
model.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 128)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
118/118 [==============================] - 94s 782ms/step - loss: 0.4658 - accuracy: 0.7462 - val_loss: 0.2717 - val_accuracy: 0.8920
Epoch 2/10
118/118 [==============================] - 86s 730ms/step - loss: 0.1553 - accuracy: 0.9417 - val_loss: 0.2009 - val_accuracy: 0.9208
Epoch 3/10
118/118 [==============================] - 87s 741ms/step - loss: 0.0846 - accuracy: 0.9699 - val_loss: 0.2543 - val_accuracy: 0.9120
Epoch 4/10
118/118 [==============================] - 87s 734ms/step - loss: 0.0402 - accuracy: 0.9870 - val_loss: 0.3526 - val_accuracy: 0.9066
Epoch 5/10
118/118 [==============================] - 88s 750ms/step - loss: 0.0345 - accuracy: 0.9868 - val_loss: 0.3870 - val_accuracy: 0.9130
Epoch 6/10
118/118 [==============================] - 88s 749ms/step - loss: 0.0124 - accuracy: 0.9963 - val_loss: 0.4319 - val_accuracy: 0.9128
Epoch 7/10
118/118 [==============================] - 88s 750ms/step - loss: 0.0122 - accuracy: 0.9958 - val_loss: 0.9231 - val_ac